## 5. Point Registration

Point-based registration allows us to help the registration via pre-defined sets of corresponding points. The 'CorrespondingPointsEuclideanDistanceMetric' minimises the distance of between a points on the fixed image and corresponding points on the moving image. The metric can be used to help in a difficult registration task by taking into account positions are known to correspond. Think of it as a way of embedding expert knowledge in the registration procedure. We can manually select points or automatically them via centroids of segmentations for example. Anything is possible.

### Registration

In [2]:
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk

In order for 3D registration to work with a point set, the 'CorrespondingPointsEuclideanDistanceMetric', should be set as metric. For the 3D case, this means that the metric should be a multimetric with the first metric of type AdvancedImageToImageMetric and the second the 'CorrespondingPointsEuclideanDistanceMetric'. The Registration parameter should therefore be set to 'MultiMetricMultiResolutionRegistration', to allow a multimetric parameter.

In [7]:
# Import Images
fixed_image = itk.imread('data/CT_3D_lung_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_3D_lung_moving.mha', itk.F)

# Import and adjust Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
parameter_map_rigid['Registration'] = [
    'MultiMetricMultiResolutionRegistration']
original_metric = parameter_map_rigid['Metric']
parameter_map_rigid['Metric'] = [original_metric[0],
                                 'CorrespondingPointsEuclideanDistanceMetric']
parameter_object.AddParameterMap(parameter_map_rigid)

The point sets do not need to be imported first, so their file name + path can be given directly to elastix. 

Registration can either be done in one line with the registration function...

In [10]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    fixed_point_set_file_name='data/CT_3D_lung_fixed_point_set.txt',
    moving_point_set_file_name='data/CT_3D_lung_moving_point_set.txt',
    log_to_console=False,
    parameter_object=parameter_object)

.. or by initiating an elastix image filter object.

In [11]:
# Load Elastix Image Filter Object
elastix_object = itk.itkElastixRegistrationMethodPython.itkElastixRegistrationMethodIF3IF3.New()
elastix_object.SetFixedImage(fixed_image)
elastix_object.SetMovingImage(moving_image)
elastix_object.SetFixedPointSetFileName('data/CT_3D_lung_fixed_point_set.txt')
elastix_object.SetMovingPointSetFileName('data/CT_3D_lung_moving_point_set.txt')
elastix_object.SetParameterObject(parameter_object)
elastix_object.SetLogToConsole(False)

# Update filter object (required)
elastix_object.UpdateLargestPossibleRegion()

# Results of Registration
result_image = elastix_object.GetOutput()
result_transform_parameters = elastix_object.GetTransformParameterObject()